![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Retrieval Optimizer

This notebook will walk through how to get going optimizing your vector retrieval.

<a href="https://colab.research.google.com/github/redis-applied-ai/retrieval-optimizer/blob/main/examples/getting_started/retrieval_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Grab data (if colab)

In [ ]:
!git clone https://github.com/redis-applied-ai/retrieval-optimizer.git temp_repo
!mv temp_repo/examples/getting_started/data .
!mv temp_repo/examples/getting_started/schema .
!mv temp_repo/examples/getting_started/study_config.yaml .
!mv temp_repo/optimize .
!mv temp_repo/images/
!rm -rf temp_repo

## 1 - Load dataset

For the retrieval optimization framework to run it requires a set of raw chunks and a set of queries with labeled chunks. The labeling tool can be used to generate these artifacts from an existing index or they can be created manually. See [README.md](../../README.md) for more information or checkout [populate_index.ipynb](./populate_index.ipynb).

For this demo, examples have been provided in the [data/](./data) directory. Let's see what they look like:

In [1]:
import json

with open("data/mazda_chunks.json", "r") as f:
    chunks = json.load(f)

chunks[0]

{'item_id': '119',
 'text': 'qMirror Defrosterí To turn on the mirror defrosters, turn the ignition switch to the ON position and push the rear window defroster switch (page 5-58).\n\nForm No.8Y64-EA-08A\n\nBlack plate (181,1)\n\nDriving Your Mazda Switches and Controls\n\nHorn\n\nTo sound the horn, press the the steering wheel.\n\nmark on\n\níSome models.\n\n5-59\n\nMazda3_8Y64-EA-08A_Edition1 Page182 Tuesday, November 27 2007 9:1 AM\n\nDriving Your Mazda Switches and Controls\n\nHazard Warning Flasher\n\nThe hazard warning lights should always be used when you stop on or near a roadway in an emergency.\n\nThe hazard warning lights warn other drivers that your vehicle is a traffic hazard and that they must take extreme caution when near it.\n\nHazard warning flasher\n\nDepress the hazard warning flasher and all the turn signals will flash.\n\nNOTE l The turn signals do not work when the\n\nhazard warning lights are on.\n\nl Check local regulations about the use of\n\nhazard warning li

In [2]:
with open("data/mazda_labeled.json", "r") as f:
    labeled = json.load(f)

labeled[0]

{'query': 'At what speed should I shift from 2 to 3 with a manual transmission?',
 'relevant_item_ids': ['86']}

#### Notes on the data

These jsons were generated using the labeling tool (see readme) however to run a study all you need to provide is a set of chunks of the form:

```json
[
    {
        "item_id": "chunk:123",
        "text": "raw content of the chunk"
    }
]
```

And a set of labeled queries:

```json
[
    {
        "query": "relevant question?",
        "relevant_item_ids": ["chunk:123"] // list of the relevant chunks to that question
    }
]
```

## 2 - python dependencies & env setup

This project utilizes poetry for dependency management. If you like, in the project root you can run `poetry install` then set the kernel version of this notebook to point to the path of the python interpreter poetry is referencing. This can be accomplish by running `poetry env info --path` after installing and setting as your kernel for the notebook.

Alternatively you can run the following line to import in a notebook native way for convenience.

In [3]:
%pip install numpy redisvl sentence-transformers redis optuna pandas nest-asyncio


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Important!

The following code allows for us to run async functions in jupyter and adds the optimize module to path for easy import.

In [7]:
import sys
import os

# Add the parent directory to the Python path to get optimize module
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

import nest_asyncio

# Apply the nest_asyncio patch: let's us run async code in Jupyter
nest_asyncio.apply()

## 3 - Define the study config

Under the hood, this framework is performing bayesian optimization utilizing the optuna library. This process determines the set of decision variables that best optimize the cost function. It accomplishes this by trying different combinations of the decision variables and intelligently traversing the solution space in the direction of best results.

At current the available decision variables are:

- algorithm ("hnsw" or "flat")
  - if hnsw secondary hyper-parameters of ef_runtime, ef_construction, and m can also be optimized.
- vector data type
- ret k
- embedding model

Different configurations of these variables will be attempted in order to **maximize** the following equation: `obj = w0 * f1 + w1 * norm_embedding_latency + w2 * norm_indexing_time` expressed in vector notation this is the dot product of `weights * metrics`. If there are additional metrics you would like to optimize over feel free to open an issue and we can look into adding it!

#### study_config.yaml
```yaml
input_data_type: "json" # currently only json but considering alternatives
raw_data_path: "data/mazda_chunks.json" # path to data files for easy read
labeled_data_path: "data/mazda_labeled.json" # labeled data

# optimization config
n_trials: 20
n_jobs: 1

# metrics to be used in objective function
metrics: ["f1_at_k", "embedding_latency", "total_indexing_time"] 
weights: [1, 1, 1] # weight of each metric respectively 

# optimization decision variables
algorithms: ["flat", "hnsw"] # indexing algorithms variables
vector_data_types: ["float32", "float16"] # vector type of indexed vectors
ret_k: [1, 10] # potential range of value to be sampled during study

# ef_runtime, ef_construction, and m only relevant if HNSW indexing
ef_runtime: [10, 20, 30, 50]
ef_construction: [100, 150, 200, 250, 300]
m: [8, 16, 64]

# embedding models to be used
embedding_models:
  - provider: "hf"
    model: "sentence-transformers/all-MiniLM-L6-v2"
    dim: 384
  - provider: "hf"
    model: "intfloat/e5-large-v2"
    dim: 1024
```

#### Load the study config


In [8]:
from optimize.study import load_config

study_config = load_config("study_config.yaml") # Loads yaml into pydantic model

study_config.embedding_models[1]

EmbeddingModel(provider='hf', model='intfloat/e5-large-v2', dim=1024)

# 4 - running a redis instance

For the following code to work we need to have a running instance of redis. 

## Running notebook local use docker:

`docker run -d --name redis -p 6379:6379 -p 8001:8001 redis/redis-stack:latest`

## Running in colab use the following shell command:

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

# 5 - Initialize the eval class

The optimization works by instantiating different variation of the eval class and running calc metrics. Let's run an evaluation to get a baseline for our study.

In [9]:
from optimize.eval import Eval

e = Eval(
    model_provider="hf",
    model_str="intfloat/e5-large-v2",
    embedding_dim=1024,
    raw_data_path=study_config.raw_data_path,
    labeled_data_path=study_config.labeled_data_path,
    input_data_type=study_config.input_data_type,
    vector_data_type=study_config.vector_data_types[0],
    algorithm=study_config.algorithms[0],
    ret_k=4,
    find_threshold=False,
    redis_url="redis://localhost:6379/0",
)

2024-11-27 11:22:45,220 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2
2024-11-27 11:22:49,807 - INFO - Use pytorch device_name: mps


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

2024-11-27 11:23:07,099 - INFO - Indexing data...
2024-11-27 11:23:07,172 - INFO - Data indexed. self.total_indexing_time=60.57899856567383s


## What just happened?

1. eval class embedded the chunks with the specified model
2. created a search index in redis
3. loaded the chunks into the index

#### Now let's calculate some metrics!

In [10]:
e.calc_metrics()

2024-11-27 11:23:14,411 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2
2024-11-27 11:23:17,085 - INFO - Use pytorch device_name: mps


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-11-27 11:23:20,358 - INFO - Calculating retrieval metrics for test_id: 3f3b8b52-0aa2-4f22-b58a-712d24cbe735
2024-11-27 11:23:20,368 - INFO - Overall f1 at 4 for retrieval: 0.5003968253968254


#### What just happened?

- the queries provided in the label section were run against the index and the result recorded
- precision, recall, and f1 @k were calculated for the configuration
- results and metadata of the eval was persisted to redis

For each test a UUID is created and information stored in redis. The following shows the test_id and the overall f1 score

In [11]:
e.settings.test_id, e.f1_at_k, e.avg_query_latency

('3f3b8b52-0aa2-4f22-b58a-712d24cbe735',
 0.5003968253968254,
 0.008097648620605469)

Note: if you open [Redis Insight](https://redis.io/insight/) you can see the full results of this test and the distance samples in detail.

# 6 - running an optimization

Now that we understand the components let's run a study.

In [12]:
from optimize.study import run_study

run_study(study_config)

[I 2024-11-27 11:23:55,319] A new study created in memory with name: test
2024-11-27 11:23:55,320 - INFO - Running for Retriever: DefaultQueryRetriever with additional_schema_fields=None




 Running for: 
 model_str: intfloat/e5-large-v2 
 algorithm: flat


2024-11-27 11:24:01,076 - INFO - Load pretrained SentenceTransformer: intfloat/e5-large-v2
2024-11-27 11:24:04,135 - INFO - Use pytorch device_name: mps


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-11-27 12:03:53,654 - INFO - Calculating retrieval metrics for test_id: 92739ff0-c188-49b1-b108-39dac290ae67
2024-11-27 12:03:53,666 - INFO - Overall f1 at 5 for retrieval: 0.5774110149110149
[I 2024-11-27 12:03:53,669] Trial 19 finished with value: 0.7266295668396562 and parameters: {'model_info': {'provider': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384}, 'algorithm': 'hnsw', 'var_dtype': 'float32', 'ret_k': 5, 'ef_runtime': 20, 'ef_construction': 200, 'm': 16}. Best is trial 14 with value: 1.0108533762611567.


Completed Bayesian optimization... 


Best Configuration: 14: {'model_info': {'provider': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384}, 'algorithm': 'hnsw', 'var_dtype': 'float32', 'ret_k': 3, 'ef_runtime': 20, 'ef_construction': 250, 'm': 8}:


Best Score: [1.0108533762611567]




## Success!

From the above we can see the overall best configuration and the settings used this is the config that best optimizes the cost function.

For a more complete view of our results let's grab the metric outputs of our study and load into a pandas dataframe for our review.

In [ ]:
from redis import Redis
import pandas as pd
study_key = f"study:{study_config.study_id}"

redis_client = Redis.from_url("redis://localhost:6379/0")

res = redis_client.json().get(study_key)

res_df = pd.DataFrame(res)

In [ ]:
res_df.sort_values(["f1@k", "embedding_latency", "indexing_time", "avg_query_latency"], ascending=False)

,ret_k,algorithm,ef_construction,ef_runtime,m,distance_metric,vector_data_type,model,model_dim,f1@k,embedding_latency,indexing_time,avg_query_latency
14,3,flat,0,0,0,cosine,float16,sentence-transformers/all-MiniLM-L6-v2,384,0.660317,0.790267,121.466003,0.017847
12,3,hnsw,100,10,64,cosine,float16,sentence-transformers/all-MiniLM-L6-v2,384,0.660317,0.748799,498.085999,0.041085
16,3,hnsw,100,10,64,cosine,float16,sentence-transformers/all-MiniLM-L6-v2,384,0.660317,0.737825,522.992981,0.022065
11,3,hnsw,100,10,64,cosine,float16,sentence-transformers/all-MiniLM-L6-v2,384,0.660317,0.721294,136.960007,0.016554
13,3,hnsw,100,10,64,cosine,float16,sentence-transformers/all-MiniLM-L6-v2,384,0.660317,0.698574,539.539001,0.008036
8,2,hnsw,200,50,64,cosine,float16,sentence-transformers/all-MiniLM-L6-v2,384,0.650000,0.736955,149.455002,0.017295
15,4,hnsw,100,10,64,cosine,float16,sentence-transformers/all-MiniLM-L6-v2,384,0.593651,0.743608,604.752991,0.007637
17,4,hnsw,100,10,64,cosine,float16,sentence-transformers/all-MiniLM-L6-v2,384,0.593651,0.739116,561.828979,0.021641
5,4,hnsw,150,30,8,cosine,float16,sentence-transformers/all-MiniLM-L6-v2,384,0.593651,0.685177,338.588989,0.007850
19,5,hnsw,100,10,64,cosine,float16,sentence-transformers/all-MiniLM-L6-v2,384,0.577411,0.737352,147.076996,0.018551


In [ ]:
res_df.groupby(["algorithm"])[["avg_query_latency", "indexing_time"]].mean()

,avg_query_latency,indexing_time
algorithm,,
flat,0.026202,106.361198
hnsw,0.022195,303.367729
